# SQL

### Задания

- Посчитайте количество книг, выпущенных после 1 января 2000 года;
- Посчитайте количество пользовательских обзоров и среднюю оценку для каждой книги;
- Определите издательство, которое издало наибольшее число книг толще 50 страниц (так вы исключите из анализа различные брошюры);
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более пользовательскими оценками;
- Посчитайте среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Содержание <a id='toc'>

[1. Цели исследования](#1)<br>
[2. Исследование таблиц](#2)<br>
[3. Задачи и выводы](#3)<br>
[3.1 Количество книг, выпущенных после 1 января 2000 года](#31)<br>
[3.2 Количество пользовательских обзоров и средняя оценка для каждой книги](#32)<br>
[3.3 Издательство с книгами больше 50 страниц](#33)<br>
[3.4 Автор с самой высокой средней оценкой книг](#34)<br>
[3.5 Среднее количество текстовых обзоров пользователей](#35)<br>
[4. Выводы](#4)

## 1. Цели исследования <a id='1'>

Стартап книжного приложения. Необходимо определить наиболее выгодные для нового магазина продукты опираясь на данные конкурентов.

## 2. Исследование таблиц <a id='2'>

Импортирование библиотек и подключение к базе данных:

In [1]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432,
                'db': 'data-analyst-final-project-db'}
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], db_config['pwd'],
db_config['host'], db_config['port'], db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
def SQL(query): 
    return pd.io.sql.read_sql(query, con = engine);

Ознакомимся с таблицами(выведем первые строки):

In [3]:
all_tables = ('Books', 
              'Authors',
              'Publishers',
              'Ratings',
              'Reviews')
for table_name in all_tables:
    print(table_name)
    display(SQL('''SELECT * FROM {} LIMIT 1'''.format(table_name)))

Books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


Authors


,author_id,author
0,1,A.S. Byatt


Publishers


,publisher_id,publisher
0,1,Ace


Ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


Reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


## 3. Задачи и выводы <a id='3'>

### 3.1 Количество книг, выпущенных после 1 января 2000 года <a id='31'>

In [4]:
SQL('''
SELECT COUNT(*) as books_count
FROM Books 
WHERE publication_date > '01-01-2000'
''')

,books_count
0,819


### 3.2 Количество пользовательских обзоров и средняя оценка для каждой книги <a id='32'>

In [5]:
SQL('''
Select * from books where title='Memoirs of a Geisha'
''')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Посчитаем количество пользовательских обзоров и среднюю оценку для каждой книги:

In [6]:
SQL('''
SELECT 
title,
rv.book_id,
COUNT(review_id) AS reviews_count,
AVG(rating) AS average_rating
FROM reviews rv JOIN ratings rt ON rv.book_id = rt.book_id
JOIN books bk ON bk.book_id = rt.book_id
GROUP BY  title, rv.book_id
ORDER BY 3 DESC
''')

,title,book_id,reviews_count,average_rating
0,Twilight (Twilight #1),948,1120,3.662500
1,The Hobbit or There and Back Again,750,528,4.125000
2,The Catcher in the Rye,673,516,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,302,492,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,299,480,4.287500
...,...,...,...,...
989,Merrick (The Vampire Chronicles #7),431,2,4.000000
990,The Adventures of Tom Sawyer and Adventures of...,625,2,5.000000
991,Everyday Italian: 125 Simple and Delicious Rec...,226,2,3.500000
992,Wicked: The Grimmerie,980,2,3.500000


### 3.3 Издательство с книгами больше 50 страниц <a id='33'>

Определим издательство, которое издало наибольшее число книг толще 50 страниц:

In [7]:
SQL('''
SELECT pb.publisher, count as books_count
FROM
    (SELECT publisher_id, COUNT(*)
    FROM books
    WHERE num_pages >50
    GROUP BY publisher_id) b_max JOIN publishers pb ON b_max.publisher_id = pb.publisher_id
WHERE count = (SELECT MAX(count) FROM (SELECT publisher_id, COUNT(*)
                                        FROM books
                                        GROUP BY publisher_id) b_cnt)
''')

,publisher,books_count
0,Penguin Books,42


In [8]:
SQL('''
SELECT pb.publisher, count as books_count
FROM
    (SELECT publisher_id, COUNT(*)
    FROM books
    WHERE num_pages >50
    GROUP BY publisher_id) b_max JOIN publishers pb ON b_max.publisher_id = pb.publisher_id
ORDER BY 2 DESC
LIMIT 1
    ''')

,publisher,books_count
0,Penguin Books,42


### 3.4 Автор с самой высокой средней оценкой книг<a id='34'>

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более пользовательскими оценками:

In [43]:
SQL('''
SELECT author, avg
FROM authors a JOIN (SELECT author_id, AVG(avg_rating)
                    FROM books bk JOIN (SELECT book_id, AVG(rating) AS avg_rating
                                        FROM ratings
                                        GROUP BY book_id
                                        HAVING COUNT(rating) >= 50) rt ON bk.book_id = rt.book_id
                    GROUP BY author_id
                    ORDER BY 2 DESC
                    LIMIT 1) top ON a.author_id = top.author_id

''')

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


### 3.5 Среднее количество текстовых обзоров пользователей<a id='35'>

Посчитаем среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок:

In [26]:
SQL('''
SELECT AVG(text_cnt)
FROM (SELECT username, COUNT(rating)
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating) > 50) rt
                JOIN (SELECT username, COUNT(text) AS text_cnt
                        FROM reviews
                        GROUP BY username) txt
                ON rt.username = txt.username
''')

,avg
0,24.333333


### 4. Выводы

- После 1 января 2000 года выпущена **821** книга;
- В датасете представленно большое количество книг с обзорами и хорошими рейтинтами, что позволит выбрать большое количество бестселеров;
- Издательство, которое издало наибольшее число книг - это **Penguin Books**, и они выпустили **42** книги;
- Автор с самой высокой средней оценкой книг - это **Diana Gabaldon**, средняя оценка ее книг **4.3**;
- Cреднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок составляет около **24** обзора.

[К содержанию](#toc)